# Automação de Indicadores

### Criando um Projeto Completo de forma a gerar indicadores e enviar OnePage via e-mails para gerentes das 25 lojas.

Um One Page é um resumo muito simples e direto ao ponto, usado pela equipe de gerência de loja para saber os principais indicadores de cada loja e permitir em 1 página (daí o nome OnePage) tanto a comparação entre diferentes lojas, quanto quais indicadores aquela loja conseguiu cumprir naquele dia ou não.


### Arquivos e Informações Importantes

- Arquivo Emails.xlsx com o nome, a loja e o e-mail de cada gerente. 

- Arquivo Vendas.xlsx com as vendas de todas as lojas. Obs: Cada gerente só deve receber o OnePage e um arquivo em excel em anexo com as vendas da sua loja. As informações de outra loja não devem ser enviados ao gerente que não é daquela loja.

- Arquivo Lojas.csv com o nome de cada Loja

- Ao final, sua rotina deve enviar ainda um e-mail para a diretoria (informações também estão no arquivo Emails.xlsx) com 2 rankings das lojas em anexo, 1 ranking do dia e outro ranking anual. Além disso, no corpo do e-mail, deve ressaltar qual foi a melhor e a pior loja do dia e também a melhor e pior loja do ano. O ranking de uma loja é dado pelo faturamento da loja.

- As planilhas de cada loja devem ser salvas dentro da pasta da loja com a data da planilha, a fim de criar um histórico de backup

### Indicadores do OnePage

- Faturamento -> Meta Ano: 1.650.000 / Meta Dia: 1000
- Diversidade de Produtos (quantos produtos diferentes foram vendidos naquele período) -> Meta Ano: 120 / Meta Dia: 4
- Ticket Médio por Venda -> Meta Ano: 500 / Meta Dia: 500

Obs: Cada indicador deve ser calculado no dia e no ano. O indicador do dia deve ser o do último dia disponível na planilha de Vendas (a data mais recente)

### Importando as Bibliotecas

In [194]:
import pandas as pd
import os
from pathlib import Path
import win32com.client as win32     # pip install pywin32  Biblioteca para enviar email Outlook


In [41]:
caminho = os.getcwd()
print(caminho)

# Importação da base de dados
vendas_df = pd.read_excel(rf'{caminho}\Bases de Dados\Vendas.xlsx')
lojas_df = pd.read_csv(rf'{caminho}\Bases de Dados\Lojas.csv', sep=';', encoding='latin1')
emails_df = pd.read_excel(rf'{caminho}\Bases de Dados\Emails.xlsx')
vendas_df = vendas_df.merge(lojas_df, on='ID Loja')
vendas_df = vendas_df.merge(emails_df, on='Loja')
display(vendas_df)


c:\Users\Michele\PycharmProjects\testesPython\Projeto1_AutomacaoProcessos


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja,Gerente,E-mail
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
...,...,...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping,Heloisa,michele.fenix.17+heloisa@gmail.com
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping,Heloisa,michele.fenix.17+heloisa@gmail.com
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping,Heloisa,michele.fenix.17+heloisa@gmail.com
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping,Heloisa,michele.fenix.17+heloisa@gmail.com


In [42]:
print(vendas_df.info())
print('-'*60)
print(vendas_df.iloc[0])    # Analisando a primeiroa linha para entender os dados se estão corretos.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100999 entries, 0 to 100998
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Código Venda    100999 non-null  int64         
 1   Data            100999 non-null  datetime64[ns]
 2   ID Loja         100999 non-null  int64         
 3   Produto         100999 non-null  object        
 4   Quantidade      100999 non-null  int64         
 5   Valor Unitário  100999 non-null  int64         
 6   Valor Final     100999 non-null  int64         
 7   Loja            100999 non-null  object        
 8   Gerente         100999 non-null  object        
 9   E-mail          100999 non-null  object        
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 8.5+ MB
None
------------------------------------------------------------
Código Venda                                      1
Data                            2019-01-01 00:00:00
ID Loja       

### Criando pastas e separando os arquivos por Loja em csv

In [160]:
ultimo_dia_vendas = str(vendas_df['Data'].max()).split(' ')[0].replace('-','_')[5:]     # Pega o ultimo dia de venda para incluir no nome do arquivo.
pasta = Path(caminho + r'\Backup Arquivos Lojas')

for loja in vendas_df['Loja'].unique():     # Pego o nome das lojas de forma única
    nova_pasta = pasta / loja   # Nome da Pasta incluindo o nome da Loja
    filtro = vendas_df.loc[vendas_df['Loja'] == loja, ]   # Filtro por Loja
    if not nova_pasta.is_dir(): # Se a pasta não existe, cria a pasta
        nova_pasta.mkdir()
    if not (nova_pasta / Path(f'{ultimo_dia_vendas}_{loja}.csv')).exists(): # Se o arquivo não existe ele acessa o caminho da nova pasta e cria o arquivo csv com o filtro do Dataframe
        filtro.to_csv(rf'{nova_pasta}\{ultimo_dia_vendas}_{loja}.csv', sep=';', index=False) 
    else:           
        print('Existe') 

### Realizando análise dos indicadores

In [161]:
# Indicadores
meta_produtos_vendidos_diario = 4
meta_produtos_vendidos_anual = 120
meta_anual = 1650000
meta_diaria = 1000
meta_ticket_medio_diario = 500
meta_ticket_medio_anual = 500

### Pegando o ultimo dia do ano

In [197]:
ultimo_dia_vendas2 = str(vendas_df['Data'].max()).split(' ')[0]
print(ultimo_dia_vendas2)

2019-12-26


### Criando dicionário de Lojas

In [193]:
dicionario_lojas = {}

for loja in lojas_df['Loja']:
    dicionario_lojas[loja] = vendas_df.loc[vendas_df['Loja']== loja, ]

display(dicionario_lojas['Iguatemi Esplanada'])

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja,Gerente,E-mail
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
...,...,...,...,...,...,...,...,...,...,...
3975,69899,2019-12-25,1,Sapato Xadrez,4,368,1472,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
3976,69903,2019-12-25,1,Terno Liso,5,720,3600,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
3977,69928,2019-12-25,1,Meia,1,30,30,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
3978,69944,2019-12-25,1,Gorro Xadrez,5,85,425,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com


### Lógica

In [247]:
for loja in lojas_df['Loja']:
    # Filtro por Loja específica
    vendas_por_loja = vendas_df.loc[vendas_df['Loja'] == loja, ]
    vendas_dia_especifico = vendas_por_loja.loc[vendas_por_loja['Data'] == ultimo_dia_vendas2]
    display(vendas_por_loja)
    display(vendas_dia_especifico)

    # - Diversidade de Produtos (quantos produtos diferentes foram vendidos naquele período) -> Meta Ano: 120 / Meta Dia: 4
    diversidade_diaria = len(vendas_dia_especifico['Produto'].unique())
    diverdidade_anual = len(vendas_por_loja['Produto'].unique())
    print(diversidade_diaria, diverdidade_anual)

    # - Faturamento -> Meta Ano: 1.650.000 / Meta Dia: 1000
    faturamento_diario = vendas_dia_especifico['Valor Final'].sum()
    faturamento_anual = vendas_por_loja['Valor Final'].sum()
    print(faturamento_diario, faturamento_anual)

    # # - Ticket Médio por Venda -> Meta Ano: 500 / Meta Dia: 500
    total_venda_dia = vendas_dia_especifico.groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_diario = total_venda_dia['Valor Final'].mean()
    
    total_venda_ano = vendas_por_loja.groupby('Código Venda').sum(numeric_only=True)
    tiket_medio_anual = total_venda_ano['Valor Final'].mean()
    print('ticket_medio_diario', ticket_medio_diario)
    print('tiket_medio_anual', tiket_medio_anual)

    # VALIDAÇÃO INDICADORES (META)
    diversidade_qtde_cor_dia = 'green' if diversidade_diaria >= meta_produtos_vendidos_diario else 'red'
    diversidade_qtde_cor_ano = 'green' if diverdidade_anual >= meta_produtos_vendidos_anual else 'red'

    faturamento_cor_dia = 'green' if faturamento_diario >= meta_diaria else 'red'
    faturamento_cor_ano = 'green' if faturamento_anual >= meta_anual else 'red'

    ticket_medio_cor_dia = 'green' if ticket_medio_diario >= meta_ticket_medio_diario else 'red'
    ticket_medio_cor_ano = 'green' if tiket_medio_anual >= meta_ticket_medio_anual else 'red'

    
    # criar a integração com o outlook
    outlook = win32.Dispatch('outlook.application')

    # criar um email
    email = outlook.CreateItem(0)

    # configurar as informações do seu e-mail
    email.To = vendas_por_loja['E-mail'].unique()[0]   # E-mail do destinatário
    email.Subject = f'OnePage do dia {ultimo_dia_vendas}, Loja: {loja}'     # Assunto do E-mail
    nome_gerente = vendas_por_loja['Gerente'].unique()[0]
    # email.Body = "Mensagem do corpo do e-mail"
    email.HTMLBody =  f'''
    <p> Bom dia, {nome_gerente}</p>

    <p>O resultado do dia {ultimo_dia_vendas} da Loja {loja} foi:</p>

    <table>
        <thead>
            <tr> 
                <td><b>Indicador</b></td>
                <td><b>Valor Dia</b></td>
                <td><b>Meta Dia</b></td>
                <td><b>Cenário Dia</b></td>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td style="text-align: center">Faturamento</td>
                <td style="text-align: center">{faturamento_diario:,.2f}</td>
                <td style="text-align: center">{meta_diaria:,.2f}</td>
                <td style="text-align: center"><font color="{faturamento_cor_dia}">◙</font></td>
            </tr>
            <tr>
                <td style="text-align: center">Diversidade</td>
                <td style="text-align: center">{diversidade_diaria}</td>
                <td style="text-align: center">{meta_produtos_vendidos_diario}</td>
                <td style="text-align: center"><font color="{diversidade_qtde_cor_dia}">◙</font></td>
            </tr>
            <tr>
                <td style="text-align: center">Ticket Médio</td>
                <td style="text-align: center">{ticket_medio_diario:,.2f}</td>
                <td style="text-align: center">{meta_ticket_medio_diario:,.2f}</td>
                <td style="text-align: center"><font color="{ticket_medio_cor_dia}">◙</font></td>
            </tr>
        </tbody>
    </table>
    <table>
        <thead>
            <tr> 
                <td><b>Indicador</b></td>
                <td><b>Valor Ano</b></td>
                <td><b>Meta Ano</b></td>
                <td><b>Cenário Ano</b></td>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td style="text-align: center">Faturamento</td>
                <td style="text-align: center">{faturamento_anual:,.2f}</td>
                <td style="text-align: center">{meta_anual:,.2f}</td>
                <td style="text-align: center"><font color="{faturamento_cor_ano}">◙</font></td>
            </tr>
            <tr>
                <td style="text-align: center">Diversidade</td>
                <td style="text-align: center">{diverdidade_anual}</td>
                <td style="text-align: center">{meta_produtos_vendidos_anual}</td>
                <td style="text-align: center"><font color="{diversidade_qtde_cor_ano}">◙</font></td>
            </tr>
            <tr>
                <td style="text-align: center">Ticket Médio</td>
                <td style="text-align: center">{tiket_medio_anual:,.2f}</td>
                <td style="text-align: center">{meta_ticket_medio_anual:,.2f}</td>
                <td style="text-align: center"><font color="{ticket_medio_cor_ano}">◙</font></td>
            </tr>
        </tbody>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes</p>
    <p>Qualquer dúvida, estou a disposição.</p>


    <p>Att.,</p>
    <p>Fulano</p>


    '''
    attachment  = pasta / loja / f'{ultimo_dia_vendas}_{loja}.csv'
    print(attachment)

    email.Attachments.Add(str(attachment))  # Para adicionar tem que transformar em string

    email.Send()   # Enviar o e-mail
    print('Email enviado para {} referente a loja {}'.format(nome_gerente, loja))
    

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja,Gerente,E-mail
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
...,...,...,...,...,...,...,...,...,...,...
3975,69899,2019-12-25,1,Sapato Xadrez,4,368,1472,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
3976,69903,2019-12-25,1,Terno Liso,5,720,3600,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
3977,69928,2019-12-25,1,Meia,1,30,30,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com
3978,69944,2019-12-25,1,Gorro Xadrez,5,85,425,Iguatemi Esplanada,Helena,michele.fenix.17+helena@gmail.com


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja,Gerente,E-mail


0 120
0 1699681
ticket_medio_diario nan
tiket_medio_anual 795.3584464202153
c:\Users\Michele\PycharmProjects\testesPython\Projeto1_AutomacaoProcessos\Backup Arquivos Lojas\Iguatemi Esplanada\12_26_Iguatemi Esplanada.csv
Email enviado para Helena referente a loja Iguatemi Esplanada


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja,Gerente,E-mail
36382,12,2019-01-02,2,Short Xadrez,4,100,400,Shopping Midway Mall,Alice,michele.fenix.17+alice@gmail.com
36383,12,2019-01-02,2,Sunga Liso,2,114,228,Shopping Midway Mall,Alice,michele.fenix.17+alice@gmail.com
36384,33,2019-01-02,2,Camisa,1,100,100,Shopping Midway Mall,Alice,michele.fenix.17+alice@gmail.com
36385,33,2019-01-02,2,Meia Xadrez,3,41,123,Shopping Midway Mall,Alice,michele.fenix.17+alice@gmail.com
36386,48,2019-01-02,2,Cueca Linho,2,93,186,Shopping Midway Mall,Alice,michele.fenix.17+alice@gmail.com
...,...,...,...,...,...,...,...,...,...,...
40214,69914,2019-12-25,2,Short Liso,2,108,216,Shopping Midway Mall,Alice,michele.fenix.17+alice@gmail.com
40215,69914,2019-12-25,2,Cinto,5,200,1000,Shopping Midway Mall,Alice,michele.fenix.17+alice@gmail.com
40216,69919,2019-12-25,2,Cinto Xadrez,3,215,645,Shopping Midway Mall,Alice,michele.fenix.17+alice@gmail.com
40217,69929,2019-12-25,2,Meia Listrado,2,37,74,Shopping Midway Mall,Alice,michele.fenix.17+alice@gmail.com


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja,Gerente,E-mail
40218,69994,2019-12-26,2,Calça,1,170,170,Shopping Midway Mall,Alice,michele.fenix.17+alice@gmail.com


...

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja,Gerente,E-mail


0 120
0 1762419
ticket_medio_diario nan
tiket_medio_anual 822.0237873134329
c:\Users\Michele\PycharmProjects\testesPython\Projeto1_AutomacaoProcessos\Backup Arquivos Lojas\Iguatemi Campinas\12_26_Iguatemi Campinas.csv
Email enviado para Pedro referente a loja Iguatemi Campinas


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja,Gerente,E-mail
12277,5,2019-01-02,25,Calça,1,170,170,Shopping Barra,Benjamin,michele.fenix.17+benjamin@gmail.com
12278,52,2019-01-02,25,Gorro Listrado,4,97,388,Shopping Barra,Benjamin,michele.fenix.17+benjamin@gmail.com
12279,100,2019-01-02,25,Camisa Gola V Liso,3,118,354,Shopping Barra,Benjamin,michele.fenix.17+benjamin@gmail.com
12280,100,2019-01-02,25,Cinto Xadrez,4,215,860,Shopping Barra,Benjamin,michele.fenix.17+benjamin@gmail.com
12281,132,2019-01-02,25,Terno Linho,2,750,1500,Shopping Barra,Benjamin,michele.fenix.17+benjamin@gmail.com
...,...,...,...,...,...,...,...,...,...,...
16234,69908,2019-12-25,25,Meia Liso,2,38,76,Shopping Barra,Benjamin,michele.fenix.17+benjamin@gmail.com
16235,69908,2019-12-25,25,Camisa,1,100,100,Shopping Barra,Benjamin,michele.fenix.17+benjamin@gmail.com
16236,69913,2019-12-25,25,Gorro,1,80,80,Shopping Barra,Benjamin,michele.fenix.17+benjamin@gmail.com
16237,69913,2019-12-25,25,Cinto Linho,1,248,248,Shopping Barra,Benjamin,michele.fenix.17+benjamin@gmail.com


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja,Gerente,E-mail


0 120
0 1653103
ticket_medio_diario nan
tiket_medio_anual 779.0306314797361
c:\Users\Michele\PycharmProjects\testesPython\Projeto1_AutomacaoProcessos\Backup Arquivos Lojas\Shopping Barra\12_26_Shopping Barra.csv
Email enviado para Benjamin referente a loja Shopping Barra


### Análise dia a dia criando coluna Atingiu Meta como Sim ou Não

In [ ]:
# Analise Anual dia por dia
for loja in lojas_df['Loja']:
    # Filtro por Loja específica
    vendas_por_loja = vendas_df.loc[vendas_df['Loja'] == loja, ]
    display(vendas_por_loja)

    # - Diversidade de Produtos (quantos produtos diferentes foram vendidos naquele período) -> Meta Ano: 120 / Meta Dia: 4
    vendas_por_data = vendas_por_loja.groupby(['Data'])['Produto'].unique().reset_index()
    for i in range (0, len(vendas_por_data)):
        vendas_por_data.loc[i, 'Tot Prod Diferentes'] = len(vendas_por_data['Produto'].iloc[i])
        if vendas_por_data['Tot Prod Diferentes'][i] >= meta_produtos_vendidos_diario:
            vendas_por_data.loc[i, 'Atingiu Meta'] = 'Sim'
        else:
            vendas_por_data.loc[i, 'Atingiu Meta'] = 'Nao'

    print('Produtos diferentes vendidos no ano: ', len(vendas_por_loja['Produto'].unique()))
    display(vendas_por_data)

    # - Faturamento -> Meta Ano: 1.650.000 / Meta Dia: 1000
    # Faturamento por Dia
    faturamento_diario = vendas_por_loja.groupby('Data')['Valor Final'].sum().reset_index()

    for i, valor in enumerate(faturamento_diario['Valor Final']):
        if valor >= meta_diaria:
            faturamento_diario.loc[i, 'Atingiu Meta'] = 'Sim'   # Uso o .loc na linha i, coluna 'Atingiu Meta'
        else:
            faturamento_diario.loc[i, 'Atingiu Meta'] = 'Não'

    display(faturamento_diario)

    # Faturamento Anual
    faturamento_anual = vendas_por_loja['Valor Final'].sum()
    print("Faturamento Anual: R$ {:,.2f}".format(faturamento_anual))

    # - Ticket Médio por Venda -> Meta Ano: 500 / Meta Dia: 500
    ticket_medio_diario = vendas_por_loja.groupby('Data')['Valor Final'].mean().reset_index()
    for i in range(0, len(ticket_medio_diario['Valor Final'])):
        if ticket_medio_diario['Valor Final'][i] > meta_ticket_medio_diario:
            ticket_medio_diario.loc[i, 'Atingiu Meta'] = 'Sim'
        else:
            ticket_medio_diario.loc[i, 'Atingiu Meta'] = 'Nao'
    display(ticket_medio_diario)


    tiket_medio_anual = vendas_por_loja['Valor Final'].mean()
    print('Ticket Medio Anual: ', tiket_medio_anual)

### Ranking de Faturamento

In [314]:
faturamento_por_loja = vendas_df.groupby('Loja')[['Loja', 'Valor Final']].sum(numeric_only=True).reset_index()
faturamento_anual_ordenado = faturamento_por_loja.sort_values(by='Valor Final', ascending=False)    # Ordenando de forma decrescente
display(faturamento_anual_ordenado)

# Exportar excel Ranking_Anual Faturamento
faturamento_anual_ordenado.to_excel(rf'{pasta}\{ultimo_dia_vendas}_Ranking_Anual.xlsx', index=False) 

faturamento_loja_dia = vendas_df.loc[vendas_df['Data'] == ultimo_dia_vendas2, ]
fat_dia = faturamento_loja_dia.groupby('Loja')[['Loja', 'Valor Final']].sum(numeric_only=True).reset_index() # Ordenando de forma decrescente
faturamento_diario_ordenado = fat_dia.sort_values(by='Valor Final', ascending=False)
display(faturamento_diario_ordenado)

# Exportar excel Ranking_Dia Faturamento
faturamento_diario_ordenado.to_excel(rf'{pasta}\{ultimo_dia_vendas}_Ranking_Dia.xlsx', index=False) 


,Loja,Valor Final
2,Iguatemi Campinas,1762419
24,Shopping Vila Velha,1731167
0,Bourbon Shopping SP,1726110
10,Rio Mar Recife,1722766
22,Shopping SP Market,1721763
6,Palladium Shopping Curitiba,1721120
4,Norte Shopping,1711968
9,Ribeirão Shopping,1707122
3,Iguatemi Esplanada,1699681
11,Rio Mar Shopping Fortaleza,1698430


,Loja,Valor Final
7,Salvador Shopping,3950
3,Novo Shopping Ribeirão Preto,3400
1,Center Shopping Uberlândia,2651
10,Shopping Eldorado,2391
8,Shopping Center Interlagos,1582
13,Shopping Recife,1366
2,Norte Shopping,1259
14,Shopping União de Osasco,1207
15,Shopping Vila Velha,937
0,Bourbon Shopping SP,676


In [329]:
email_diretoria = [email for email in emails_df['E-mail'] if 'diretoria' in email]
print(email_diretoria[0])


# criar a integração com o outlook
outlook = win32.Dispatch('outlook.application')
# criar um email
email = outlook.CreateItem(0)

# configurar as informações do seu e-mail
email.To = email_diretoria[0] # Email Destino
email.Subject = "Ranking de Faturamento Diario e Anual Lojas"
email.Body = f'''
A Loja que obteve o maior faturamento no dia {ultimo_dia_vendas2} foi {faturamento_diario_ordenado['Loja'].values[0]} com um faturamento anual de R$ {faturamento_diario_ordenado['Valor Final'].values[0]:,.2f}
A Loja que obteve o pior faturamento no dia {ultimo_dia_vendas2} foi {faturamento_diario_ordenado['Loja'].values[-1]} com um faturamento anual de R$ {faturamento_diario_ordenado['Valor Final'].values[-1]:,.2f}

A Loja que obteve o maior faturamento anual foi {faturamento_anual_ordenado['Loja'].values[0]} com um faturamento anual de R$ {faturamento_anual_ordenado['Valor Final'].values[0]:,.2f}
A loja que obteve o pior faturamento anual foi  {faturamento_anual_ordenado['Loja'].values[-1]} com um faturamento anual de R$ {faturamento_anual_ordenado['Valor Final'].values[-1]:,.2f}
'''

anexo1 = pasta / f'{ultimo_dia_vendas}_Ranking_Dia.xlsx'
anexo2 = pasta / f'{ultimo_dia_vendas}_Ranking_Anual.xlsx'
email.Attachments.Add(str(anexo1))
email.Attachments.Add(str(anexo2))


email.Send()
print('E-mail da Diretoria enviado')


michele.fenix.17+diretoria@gmail.com
E-mail da Diretoria enviado
